# Rapids CUML

Di seguito è riportato un tentativo (fallimentare) di utilizzare la GPU per l'addestramento di modelli, sfruttando le API di [Cuml](https://docs.rapids.ai/api/cuml/stable/).
Sebbene Rapids garantisca il funzionamento su WSL2 con GPU Nvidia (grazie a [CUDA Toolkit](https://developer.nvidia.com/cuda-11-8-0-download-archive?target_os=Linux&target_arch=x86_64&Distribution=WSL-Ubuntu&target_version=2.0&target_type=deb_local)) di versione 6 o superiore, nessuno dei nostri tentativi (sia su GPU v6 che v7) è andato a buon fine. Infatti sebbene la GPU venga riconosciuta, il training dei modelli fallisce con il seguente errore

```
    RuntimeError: Fatal CUDA error encountered at: /__w/cudf/cudf/cpp/src/bitmask/null_mask.cu:93: 801 cudaErrorNotSupported operation not supported   
```

già nella funzione `cuml.model_selection.train_test_split`.

In [1]:
%pip install numpy
%pip install pandas
%pip install matplotlib
%pip install scikit-learn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
import cuml
import cudf
from cuml.model_selection._split import train_test_split
from sklearn.model_selection import train_test_split

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
file_list = ["garmin_edge_820/3993730634_ACTIVITY_data.csv",
             "garmin_edge_820/4557226804_ACTIVITY_data.csv",
             "garmin_edge_820/4593452980_ACTIVITY_data.csv",
             "garmin_edge_820/5191513011_ACTIVITY_data.csv",
]
combined_df = pd.concat([pd.read_csv(file, sep=";") for file in file_list], ignore_index=True)

In [3]:
def convert_brackets(string):
    return string.replace('[', '(').replace(']', ')')

combined_df.columns = [convert_brackets(col) for col in combined_df.columns]

In [4]:
combined_df['timestamp(s)'] = combined_df['timestamp(s)'] + 631065600
combined_df['time'] = pd.to_datetime(combined_df.pop('timestamp(s)'), unit='s')
combined_df.set_index("time", inplace=True)

In [5]:
combined_df = combined_df[combined_df['speed(m/s)'] != 0]
combined_df = combined_df.dropna(subset=['speed(m/s)'])

In [6]:
combined_df['time_since_start(s)'] = combined_df.groupby(pd.Grouper(freq='D')).cumcount() + 1

In [7]:
hr_zones = [(0, 128), (129, 146), (147, 156), (157, 165),(166, 174), (175, 179), (180, float('inf'))]
power_zones = [(0, 157), (158, 186), (187, 200), (201, 218),(219, 247), (248, 287), (288, float('inf'))]

def get_zone(rate, zones):
    for zone, (lower, upper) in enumerate(zones, start=1):
        if lower <= rate <= upper:
            return zone
        
combined_df['hr_zone'] = combined_df['heart_rate(bpm)'].apply(get_zone, zones=hr_zones)
combined_df['pwr_zone'] = combined_df['power(watts)'].apply(get_zone, zones=power_zones)

In [8]:
combined_df['altitude_diff(m)'] = combined_df['altitude(m)'] - combined_df['altitude(m)'].shift(1)
combined_df['distance_diff(m)'] = combined_df['distance(m)'] - combined_df['distance(m)'].shift(1)
combined_df[['altitude_diff(m)', 'distance_diff(m)']] = combined_df[['altitude_diff(m)', 'distance_diff(m)']].fillna(0)
combined_df['slope_percent'] = np.where(combined_df['distance_diff(m)'] == 0, 0, combined_df['altitude_diff(m)'] / combined_df['distance_diff(m)'] * 100)

In [9]:
combined_df.drop(combined_df[combined_df['heart_rate(bpm)'] < 80].index, inplace=True)

In [10]:
window_size = 3 
combined_df['avg_power(watts)'] = combined_df['power(watts)'].rolling(window=int(window_size), center=True).mean()
combined_df['avg_power(watts)'] = combined_df['avg_power(watts)'].dropna()

In [11]:
combined_df['power_left(watts)'] = combined_df['left_right_balance'] - 128
combined_df['power_right(watts)'] = 100 - combined_df['power_left(watts)']

In [12]:
combined_df = combined_df.drop(['left_power_phase(degrees)',
                            'left_power_phase_peak(degrees)',
                            'right_power_phase(degrees)',
                            'right_power_phase_peak(degrees)',
                            'left_right_balance'], axis=1)

In [13]:
combined_df['avg_power(watts)'] = combined_df['avg_power(watts)'].fillna(combined_df['avg_power(watts)'].mean())

In [14]:
cuda_df = cudf.DataFrame(combined_df)
X = cuda_df.drop('heart_rate(bpm)', axis=1)
y = cuda_df['heart_rate(bpm)']
X_train, X_test, y_train, y_test = cuml.model_selection.train_test_split(X, y, test_size=1/3, train_size=0.75,  random_state=42)

RuntimeError: Fatal CUDA error encountered at: /__w/cudf/cudf/cpp/src/bitmask/null_mask.cu:93: 801 cudaErrorNotSupported operation not supported

In [ ]:
X_cudf_train = cudf.DataFrame.from_pandas(X_train)
X_cudf_test = cudf.DataFrame.from_pandas(X_test)

y_cudf_train = cudf.Series(y_train.values)

In [ ]:
lr = cuml.LinearRegression(algorithm='svd', fit_intercept=True, normalize=True)
lr.fit(X_cudf_train,y_cudf_train)